# Train 

In [10]:
from time import time
from tqdm import tqdm
import torch
import torch.nn
import torch.optim
import numpy as np

In [11]:
import model

In [12]:
import dataSeismic as data

In [13]:
it=iter(data.train_loader)
real_batch = next(iter(it))

In [14]:
real_batch

[tensor([[[[0.6064, 0.4235, 0.4105,  ..., 0.5496, 0.4140, 0.4903],
           [0.6360, 0.5801, 0.3417,  ..., 0.4808, 0.4754, 0.6124],
           [0.7092, 0.3604, 0.6128,  ..., 0.7203, 0.4393, 0.5533],
           ...,
           [0.3623, 0.4133, 0.7859,  ..., 0.4595, 0.5451, 0.5023],
           [0.3419, 0.6374, 0.7905,  ..., 0.5519, 0.5540, 0.5074],
           [0.4537, 0.8460, 0.5481,  ..., 0.5991, 0.4715, 0.4277]]],
 
 
         [[[0.5268, 0.3711, 0.3954,  ..., 0.6562, 0.6279, 0.5548],
           [0.7201, 0.6847, 0.4831,  ..., 0.5847, 0.5988, 0.2789],
           [0.5225, 0.6502, 0.4690,  ..., 0.3547, 0.1949, 0.2636],
           ...,
           [0.4812, 0.5367, 0.5148,  ..., 0.4998, 0.5561, 0.5944],
           [0.5852, 0.7521, 0.5984,  ..., 0.4894, 0.6851, 0.5333],
           [0.8587, 0.6052, 0.4944,  ..., 0.6175, 0.5593, 0.6657]]],
 
 
         [[[0.5210, 0.5910, 0.6164,  ..., 0.6241, 0.4817, 0.5561],
           [0.4654, 0.6273, 0.6345,  ..., 0.6600, 0.7636, 0.6998],
           [0.5353

In [15]:
cinn = model.MNIST_cINN(5e-4)
cinn.cuda()
scheduler = torch.optim.lr_scheduler.MultiStepLR(cinn.optimizer, milestones=[20, 40], gamma=0.1)

N_epochs = 60
t_start = time()
nll_mean = []

In [16]:
print('Epoch\tBatch/Total \tTime \tNLL train\tNLL val\tLR')
for epoch in range(N_epochs):
    for i, (x, l) in enumerate(data.train_loader): #data.train_loader):
        x, l = x.cuda(), l.cuda()
        z, log_j = cinn(x, l)

        nll = torch.mean(z**2) / 2 - torch.mean(log_j) / model.ndim_total
        nll.backward()
        torch.nn.utils.clip_grad_norm_(cinn.trainable_parameters, 10.)
        nll_mean.append(nll.item())
        cinn.optimizer.step()
        cinn.optimizer.zero_grad()

        if not i % 50:
            with torch.no_grad():
                z, log_j = cinn(data.val_x, data.val_l) #cinn(data.val_x, data.val_l)
                nll_val = torch.mean(z**2) / 2 - torch.mean(log_j) / model.ndim_total

            print('%.3i \t%.5i/%.5i \t%.2f \t%.6f\t%.6f\t%.2e' % (epoch,
                                                            i, len(data.train_loader),
                                                            (time() - t_start)/60.,
                                                            np.mean(nll_mean),
                                                            nll_val.item(),
                                                            cinn.optimizer.param_groups[0]['lr'],
                                                            ), flush=True)
            nll_mean = []
    scheduler.step()

Epoch	Batch/Total 	Time 	NLL train	NLL val	LR
000 	00000/00086 	0.07 	0.127287	-0.148896	5.00e-04
000 	00050/00086 	0.19 	-0.876542	-1.474705	5.00e-04
001 	00000/00086 	0.27 	-1.538668	-1.675711	5.00e-04
001 	00050/00086 	0.37 	-1.619762	-1.944390	5.00e-04
002 	00000/00086 	0.44 	-1.651419	-1.966140	5.00e-04
002 	00050/00086 	0.55 	-1.676837	-1.976722	5.00e-04
003 	00000/00086 	0.63 	-1.690132	-1.996166	5.00e-04
003 	00050/00086 	0.75 	-1.707145	-2.038876	5.00e-04
004 	00000/00086 	0.82 	-1.708843	-2.020774	5.00e-04
004 	00050/00086 	0.92 	-1.723865	-2.024717	5.00e-04
005 	00000/00086 	0.99 	-1.723674	-2.063748	5.00e-04
005 	00050/00086 	1.08 	-1.732357	-2.046184	5.00e-04
006 	00000/00086 	1.15 	-1.734011	-2.053850	5.00e-04
006 	00050/00086 	1.25 	-1.742465	-2.057013	5.00e-04
007 	00000/00086 	1.33 	-1.742760	-2.067485	5.00e-04
007 	00050/00086 	1.42 	-1.750128	-1.998251	5.00e-04
008 	00000/00086 	1.50 	-1.749763	-2.033202	5.00e-04
008 	00050/00086 	1.59 	-1.760988	-2.046696	5.00e-04
0

In [17]:
torch.save(cinn.state_dict(), 'output/mnist_cinn.pt')